In [1]:
import pandas as pd

In [2]:
import Base.bp2DState
from Base.bp2DSimpleHeuristics import single_type_heuristic, first_fit, next_fit, most_enclosed_position, max_rest, \
    best_fit, first_fit_decreasing, next_fit_decreasing, get_all_heuristics, random_fit
from Base.bp2DState import State
from Base.bpReadWrite import ReadWrite
from Base.bp2DBox import Box
from Base.bpStateGenerators import random_state_generator, state_generator
import time

from Base.bp2DPlot import plot_packing_state
from Base.bp2DSimpleHeuristicsFunctions import most_enclosed_position_in_bin
from Base.bp2DState import Bin
from Base.bp2DState import Action, State
from Base.bpUtil import *
from Base.bpStateGenerators import random_state_generator
import matplotlib.pyplot as plt

In [9]:
from rectpack import SORT_AREA, PackingMode, PackingBin, newPacker

from rectpack import (MaxRectsBl, MaxRectsBssf, MaxRectsBaf, MaxRectsBlsf,
SkylineBl, SkylineBlWm, SkylineMwf, SkylineMwfl, SkylineMwfWm, SkylineMwflWm,
GuillotineBssfSas, GuillotineBssfSlas, GuillotineBssfLlas, GuillotineBssfMaxas,
GuillotineBssfMinas, GuillotineBlsfSas, GuillotineBlsfLas, GuillotineBlsfSlas, GuillotineBlsfLlas, 
                     GuillotineBlsfMaxas, GuillotineBlsfMinas, GuillotineBafSas, GuillotineBafLas,
                     GuillotineBafSlas, GuillotineBafLlas, GuillotineBafMaxas, GuillotineBafMinas)
from ShakerBFDProMax import ShakerBFDProMax
from converterFromRectPack import convert
from Base.bpReadWrite import ReadWrite
from os import path, listdir, mkdir
input_folder = './submission_datasets'
#output_folder = './results'
cases = []
for f in listdir(input_folder):
    if '5000' not in f and '2000' not in f and '10000' not in f:
        cases.append(f)
if not path.exists(output_folder):
    mkdir(output_folder)


In [12]:
df = pd.DataFrame(columns=['algo', 'packing', 'test_case', 'boxes_count'])

In [13]:
def ffd_fast(state: State, plot_result=False, plot_steps=False, plot_name=None):
    '''ffd'''
    start = time.time()
    step = 0
    name = 'ffd_plot'
    if plot_name is not None:
        name = plot_name
    sort_boxes_in_state(state)
    while state.has_open_boxes():
        box = state.get_next_open_box()
        first_fit(state, box)

        if plot_steps:
            plot_packing_state(state=state, step=step, fname=f"./vis/{name}_step")
            step += 1
    if plot_result:
        plot_packing_state(state, fname=f"./vis/{name}_result")
    state.solution_runtime = time.time() - start

In [14]:
from tqdm import tqdm
for case in tqdm(cases):
    test_case = ReadWrite.read_state(path=path.join(input_folder, case))
    ffd_fast(test_case)
    df = pd.concat([pd.DataFrame([['ffd', 'orgs bottom-left', case, len(test_case.bins)]], columns=df.columns), df], ignore_index=True)
    tqdm._instances.clear()
    #test_case = ReadWrite.read_state(path=path.join(input_folder, case))
   # if not solution.is_valid(test_case):
       # print(f'alarm, wrong solution!!! on {case}')
        #break

100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [04:36<00:00,  6.28s/it]


In [15]:
df['boxes_count'] = df['boxes_count'].astype(int)
df['packing'] = df['packing'].astype(str)

In [16]:
df.to_csv("algos_comparison_ffd_and_shake.csv", index=False)

In [19]:
df.head(10)

,algo,packing,test_case,boxes_count
0,ffd_shaker_pro,orgs bottom-left,uniform_500,166
1,ffd_shaker_pro,orgs bottom-left,uniform_50,20
2,ffd_shaker_pro,orgs bottom-left,uniform_1000,334
3,ffd_shaker_pro,orgs bottom-left,uniform_100,36
4,ffd_shaker_pro,orgs bottom-left,small_500,20
...,...,...,...,...
58,ffd,orgs bottom-left,known_1000_bins_66,9
59,ffd,orgs bottom-left,extremes_500,253
60,ffd,orgs bottom-left,extremes_50,26
61,ffd,orgs bottom-left,extremes_1000,505


In [17]:
df.groupby('test_case').apply(lambda g:g.sort_values("boxes_count", ascending=False))

algo           packing      test_case  boxes_count
test_case                                                                     
extremes_100  41      ffd_shaker  orgs bottom-left   extremes_100           52
              20  ffd_shaker_pro  orgs bottom-left   extremes_100           51
              62             ffd  orgs bottom-left   extremes_100           51
extremes_1000 40      ffd_shaker  orgs bottom-left  extremes_1000          506
              19  ffd_shaker_pro  orgs bottom-left  extremes_1000          505
...                          ...               ...            ...          ...
uniform_50    22      ffd_shaker  orgs bottom-left     uniform_50           18
              43             ffd  orgs bottom-left     uniform_50           18
uniform_500   0   ffd_shaker_pro  orgs bottom-left    uniform_500          166
              21      ffd_shaker  orgs bottom-left    uniform_500          154
              42             ffd  orgs bottom-left    uniform_500          150

[63 rows x 4 columns]

In [15]:
df.groupby('test_case')['boxes_count'].min()

test_case
bp_1000_0.txt     74
bp_1000_1.txt    507
bp_1000_2.txt    405
bp_1000_3.txt    200
bp_50_00.txt       2
bp_50_01.txt       2
bp_50_02.txt       4
bp_50_03.txt       3
bp_50_04.txt       4
bp_50_05.txt       4
bp_50_06.txt       6
bp_50_07.txt       6
bp_50_08.txt       7
bp_50_09.txt       8
bp_50_10.txt      26
bp_50_11.txt      26
bp_50_12.txt      26
bp_50_13.txt      26
bp_50_14.txt      26
bp_50_15.txt      26
bp_50_16.txt      26
bp_50_17.txt      26
bp_50_18.txt      26
bp_50_19.txt      26
bp_50_20.txt      23
bp_50_21.txt      20
bp_50_22.txt      20
bp_50_23.txt      19
bp_50_24.txt      23
bp_50_25.txt      22
bp_50_26.txt      19
bp_50_27.txt      22
bp_50_28.txt      18
bp_50_29.txt      23
bp_50_30.txt      11
bp_50_31.txt      11
bp_50_32.txt      11
bp_50_33.txt      11
bp_50_34.txt      11
bp_50_35.txt      11
bp_50_36.txt      11
bp_50_37.txt      11
bp_50_38.txt      11
bp_50_39.txt      11
Name: boxes_count, dtype: int64

In [36]:
df[df['test_case'] == "uniform_500"]['boxes_count'].max()

158

In [80]:
df

,algo,packing,test_case,boxes_count
0,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafMinas'>,uniform_500,148
1,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafMaxas'>,uniform_500,151
2,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLlas'>,uniform_500,151
3,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafSlas'>,uniform_500,149
4,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLas'>,uniform_500,146
...,...,...,...,...
716,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBlsf'>,extremes_100,51
717,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBaf'>,extremes_100,51
718,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBssf'>,extremes_100,51
719,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBl'>,extremes_100,51


In [127]:
best_packers = [GuillotineBafSlas, GuillotineBlsfSlas, GuillotineBafMinas, GuillotineBafLas, MaxRectsBssf, MaxRectsBlsf, MaxRectsBaf]

In [29]:
df[df['test_case'] == 'small_1000'].sort_values('boxes_count')

,algo,packing,test_case,boxes_count
6,ffd_shaker_pro,orgs bottom-left,small_1000,40
48,ffd,orgs bottom-left,small_1000,40
27,ffd_shaker,orgs bottom-left,small_1000,42


In [120]:
grouped = df.groupby('test_case')

In [121]:
for_ranks = grouped.apply(lambda x: x.reset_index(drop = True)).reset_index(drop = True)
for_ranks['rank'] = grouped['boxes_count'].rank(method='min',  ascending=True)
for_ranks

,algo,packing,test_case,boxes_count,rank
0,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafMinas'>,extremes_100,51,10.0
1,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafMaxas'>,extremes_100,51,18.0
2,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLlas'>,extremes_100,51,18.0
3,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafSlas'>,extremes_100,51,12.0
4,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLas'>,extremes_100,51,1.0
...,...,...,...,...,...
562,shaker BFD pro,<class 'rectpack.skyline.SkylineBl'>,uniform_500,158,1.0
563,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBlsf'>,uniform_500,146,1.0
564,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBaf'>,uniform_500,146,1.0
565,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBssf'>,uniform_500,147,1.0


In [102]:
df.groupby(['test_case', 'packing'])['boxes_count'].count()

test_case     packing                                         
extremes_100  <class 'rectpack.guillotine.GuillotineBafLas'>      6
              <class 'rectpack.guillotine.GuillotineBafLlas'>     6
              <class 'rectpack.guillotine.GuillotineBafMaxas'>    6
              <class 'rectpack.guillotine.GuillotineBafMinas'>    6
              <class 'rectpack.guillotine.GuillotineBafSas'>      6
                                                                 ..
uniform_500   <class 'rectpack.skyline.SkylineBlWm'>              1
              <class 'rectpack.skyline.SkylineMwf'>               1
              <class 'rectpack.skyline.SkylineMwfWm'>             1
              <class 'rectpack.skyline.SkylineMwfl'>              1
              <class 'rectpack.skyline.SkylineMwflWm'>            1
Name: boxes_count, Length: 568, dtype: int64

In [46]:
grouped['boxes_count'].rank(method='min')

0      10.0
1      18.0
2      18.0
3      12.0
4       1.0
       ... 
716     1.0
717     1.0
718     1.0
719     1.0
720     1.0
Name: boxes_count, Length: 721, dtype: float64